![alternative text](../../data/colab_chatgpt.png)
![alternative text](../../data/colab2_chatgpt.png)
![alternative text](../../data/colab3_chatgpt.png)


![alternative text](../../data/cont_chatgpt.png)
![alternative text](../../data/cont2_chatgpt.png)
![alternative text](../../data/cont3_chatgpt.png)

![alternative text](../../data/whybrid_chatgpt.png)
![alternative text](../../data/shybrid_chatgpt.png)



In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Content Based Filtering 

In [10]:
# https://www.kaggle.com/datasets/shubhammehta21/movie-lens-small-latest-dataset
# Load MovieLens dataset
movies  = pd.read_csv('../../data/movies.csv')
ratings = pd.read_csv('../../data/ratings.csv')


In [126]:
# Merge movies and ratings data
movie_ratings = pd.merge(ratings, movies, on='movieId')
np.max(movie_ratings['rating'])

5.0

In [108]:
# Pivot the data to create a user-item rating matrix and change no rating to zero
user_movie_ratings = movie_ratings.pivot_table(index='userId', columns='title', values='rating').fillna(0)
user_movie_ratings

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.5,3.5,0.0,0.0,0.0


In [28]:
def cosine_similarity(vectors):
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    cosine_sim = np.dot(vectors, vectors.T) / (norms * norms.T)
    return cosine_sim

In [29]:
# Convert user-movie ratings to a numpy array
ratings_matrix = user_movie_ratings.values
print(ratings_matrix.shape)

# Calculate the cosine similarity between movies, each movie is defined as 1 x number of users feature vector 
cosine_sim = cosine_similarity(ratings_matrix.T)

# Convert cosine similarity matrix to a DataFrame
cosine_sim_df = pd.DataFrame(cosine_sim, index=user_movie_ratings.columns, columns=user_movie_ratings.columns)
cosine_sim_df


(610, 9719)


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.141653,0.000000,...,0.000000,0.342055,0.543305,0.707107,0.0,0.000000,0.139431,0.327327,0.000000,0.0
'Hellboy': The Seeds of Creation (2004),0.000000,1.000000,0.707107,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
'Round Midnight (1986),0.000000,0.707107,1.000000,0.000000,0.000000,0.0,0.176777,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
'Salem's Lot (2004),0.000000,0.000000,0.000000,1.000000,0.857493,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
'Til There Was You (1997),0.000000,0.000000,0.000000,0.857493,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ (1999),0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.211467,0.216295,0.097935,0.132489,...,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.192259,0.000000,0.170341,0.0
xXx (2002),0.139431,0.000000,0.000000,0.000000,0.000000,0.0,0.089634,0.000000,0.276512,0.019862,...,0.069716,0.305535,0.173151,0.246482,0.0,0.192259,1.000000,0.270034,0.100396,0.0
xXx: State of the Union (2005),0.327327,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.156764,0.000000,...,0.000000,0.382543,0.177838,0.231455,0.0,0.000000,0.270034,1.000000,0.000000,0.0


## Item-Based Collaborative Filtering:

    Recommend movies to a user based on similar contents to what they have already watched
    By grouping movies based on their ratings 
    Say a user watched "Toy Story (1995)", we look for movies that are most similar to it


In [32]:
def item_based_recommendations(movie_title, num_recommendations=5):
    items  = cosine_sim_df[movie_title].sort_values(ascending=False)
    idx    = items.index[1:num_recommendations+1]
    values = items[1:num_recommendations+1]
    recommended_movies = idx.tolist()
    return [(recommended_movie,value) for recommended_movie,value in zip(recommended_movies,values)]

In [33]:
recommended_movies = item_based_recommendations("Toy Story (1995)")
print("Recommended movies similar to Toy Story:")
for movie_title in recommended_movies:
    print(movie_title)


Recommended movies similar to Toy Story:
('Toy Story 2 (1999)', 0.5726012603197154)
('Jurassic Park (1993)', 0.5656368040861566)
('Independence Day (a.k.a. ID4) (1996)', 0.5642616935276659)
('Star Wars: Episode IV - A New Hope (1977)', 0.5573881705799366)
('Forrest Gump (1994)', 0.5470959079401742)


## Content Based Recommendation

In [37]:
# Extract genres as features for content-based filtering
vectorizer = CountVectorizer(tokenizer=lambda x: x.split('|'))
genre_matrix = vectorizer.fit_transform(movies['genres']).toarray()

# Calculate cosine similarity between movies based on genres
cosine_sim_genre = cosine_similarity(genre_matrix)
cosine_sim_genre_df = pd.DataFrame(cosine_sim_genre, index=movies['title'], columns=movies['title'])
cosine_sim_genre_df

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),...,Gintama: The Movie (2010),anohana: The Flower We Saw That Day - The Movie (2013),Silver Spoon (2014),Love Live! The School Idol Movie (2015),Jon Stewart Has Left the Building (2015),Black Butler: Book of the Atlantic (2017),No Game No Life: Zero (2017),Flint (2017),Bungo Stray Dogs: Dead Apple (2018),Andrew Dice Clay: Dice Rules (1991)
title,,,,,,,,,,,,,,,,,,,,,
Toy Story (1995),1.000000,0.774597,0.316228,0.258199,0.447214,0.000000,0.316228,0.632456,0.000000,0.258199,...,0.447214,0.316228,0.316228,0.447214,0.0,0.670820,0.774597,0.00000,0.316228,0.447214
Jumanji (1995),0.774597,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.816497,0.000000,0.333333,...,0.000000,0.000000,0.000000,0.000000,0.0,0.288675,0.333333,0.00000,0.000000,0.000000
Grumpier Old Men (1995),0.316228,0.000000,1.000000,0.816497,0.707107,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.353553,0.000000,0.500000,0.000000,0.0,0.353553,0.408248,0.00000,0.000000,0.707107
Waiting to Exhale (1995),0.258199,0.000000,0.816497,1.000000,0.577350,0.000000,0.816497,0.000000,0.000000,0.000000,...,0.288675,0.408248,0.816497,0.000000,0.0,0.288675,0.333333,0.57735,0.000000,0.577350
Father of the Bride Part II (1995),0.447214,0.000000,0.707107,0.577350,1.000000,0.000000,0.707107,0.000000,0.000000,0.000000,...,0.500000,0.000000,0.707107,0.000000,0.0,0.500000,0.577350,0.00000,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Black Butler: Book of the Atlantic (2017),0.670820,0.288675,0.353553,0.288675,0.500000,0.288675,0.353553,0.000000,0.500000,0.288675,...,0.750000,0.353553,0.353553,0.500000,0.0,1.000000,0.866025,0.00000,0.707107,0.500000
No Game No Life: Zero (2017),0.774597,0.333333,0.408248,0.333333,0.577350,0.000000,0.408248,0.000000,0.000000,0.000000,...,0.577350,0.408248,0.408248,0.577350,0.0,0.866025,1.000000,0.00000,0.408248,0.577350
Flint (2017),0.000000,0.000000,0.000000,0.577350,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.707107,0.707107,0.000000,0.0,0.000000,0.000000,1.00000,0.000000,0.000000


In [38]:
# Recommend movies for a user based on content
def content_based_recommendations(movie_title, num_recommendations=5):
    items  = cosine_sim_genre_df[movie_title].sort_values(ascending=False)
    idx    = items.index[1:num_recommendations+1]
    values = items[1:num_recommendations+1]
    recommended_movies = idx.tolist()
    return [(recommended_movie,value) for recommended_movie,value in zip(recommended_movies,values)]

In [40]:
recommended_movies = content_based_recommendations("Toy Story (1995)")
recommended_movies

[('Adventures of Rocky and Bullwinkle, The (2000)', 0.9999999999999998),
 ("Emperor's New Groove, The (2000)", 0.9999999999999998),
 ('Monsters, Inc. (2001)', 0.9999999999999998),
 ('Tale of Despereaux, The (2008)', 0.9999999999999998),
 ('Wild, The (2006)', 0.9999999999999998)]

## Neural Collaborative Filtering Recommendation 

    Learning latent factors in the user-item dataset, this is a hybrid approach. 

In [41]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error
from matplotlib.pylab import plt


In [234]:

# Load MovieLens 100k dataset
ratings = pd.read_csv('http://files.grouplens.org/datasets/movielens/ml-100k/u.data', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])
movies = pd.read_csv('http://files.grouplens.org/datasets/movielens/ml-100k/u.item', sep='|', encoding='latin-1', usecols=[0, 1, 2, 3, 4], names=['item_id', 'title', 'release_date', 'video_release_date', 'IMDb_URL'])

# Extract year from release date
movies['year'] = movies['release_date'].str.extract(r'(\d{4})').astype(float)

# Fill missing values
movies['year'].fillna(movies['year'].mean(), inplace=True)

# Encode genres as binary features
genres = ['unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
for genre in genres:
    movies[genre] = pd.read_csv('http://files.grouplens.org/datasets/movielens/ml-100k/u.item', sep='|', encoding='latin-1', usecols=[5+genres.index(genre)], names=[genre])

# Prepare content features
content_features = movies[['year'] + genres].values

# Convert to PyTorch tensors
user_ids = torch.tensor(ratings['user_id'].values - 1, dtype=torch.long)
item_ids = torch.tensor(ratings['item_id'].values - 1, dtype=torch.long)
ratings = torch.tensor(ratings['rating'].values, dtype=torch.float32)
content_features = torch.tensor(content_features, dtype=torch.float32)

# Split data into training and test sets
user_ids_train, user_ids_test, item_ids_train, item_ids_test, ratings_train, ratings_test = train_test_split(user_ids, item_ids, ratings, test_size=0.2, random_state=42)

class RatingDataset(torch.utils.data.Dataset):
    def __init__(self, user_ids, item_ids, ratings):
        self.user_ids = user_ids
        self.item_ids = item_ids
        self.ratings = ratings
    
    def __len__(self):
        return len(self.user_ids)
    
    def __getitem__(self, idx):
        return self.user_ids[idx], self.item_ids[idx], self.ratings[idx]

train_dataset = RatingDataset(user_ids_train, item_ids_train, ratings_train)
test_dataset = RatingDataset(user_ids_test, item_ids_test, ratings_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)



In [239]:
# Define a collaborative filtering model
class CollaborativeFiltering(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim):
        super(CollaborativeFiltering, self).__init__()
        self.user_embed = nn.Embedding(num_users, embedding_dim)
        self.item_embed = nn.Embedding(num_items, embedding_dim)
        self.fc1 = nn.Linear(2 * embedding_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    
    def forward(self, user_ids, item_ids):
        user_embeds = self.user_embed(user_ids)
        item_embeds = self.item_embed(item_ids)
        concatenated = torch.cat([user_embeds, item_embeds], dim=-1)
        x = torch.relu(self.fc1(concatenated))
        x = torch.relu(self.fc2(x))
        output = self.fc3(x)
        return output.squeeze()
    

collab_model = CollaborativeFiltering(len(user_ids), len(item_ids), embedding_dim = 20)

In [ ]:
# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(collab_model.parameters(), lr=0.0001)
num_epochs = 100
for epoch in range(num_epochs):
    for batch_user_ids, batch_item_ids, batch_ratings in train_loader:
        optimizer.zero_grad()
        batch_predictions = collab_model(batch_user_ids, batch_item_ids)
        loss = criterion(batch_predictions, batch_ratings)
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 5 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


Epoch [5/100], Loss: 1.0816
Epoch [10/100], Loss: 1.3259
Epoch [15/100], Loss: 1.1702
Epoch [20/100], Loss: 0.9031
Epoch [25/100], Loss: 0.7994
Epoch [30/100], Loss: 1.0532
Epoch [35/100], Loss: 0.9139
Epoch [40/100], Loss: 0.6916
Epoch [45/100], Loss: 0.7069
Epoch [50/100], Loss: 0.7001
Epoch [55/100], Loss: 0.7324


In [ ]:
user_id = 9
x = user_movie_ratings[user_movie_ratings.index == user_id].values 
plt.figure(figsize = (20,5))
plt.plot(x.squeeze())
item_id = np.argmax(x == 4)

In [ ]:
collab_model.eval()
with torch.no_grad():
    user_id_test = torch.tensor([user_id], dtype=torch.long)
    item_id_test = torch.tensor([item_id], dtype=torch.long)
    predicted_rating = collab_model(user_id_test, item_id_test).item()
    print(f"Predicted Rating for user 1 and item 2: {predicted_rating:.4f}")